In [1]:
# Global imports
import biosteam as bst, thermosteam as tmo

In [2]:
# Local imports
from atj_saf.atj_bst.atj_bst_chemicals import *
from atj_saf.atj_bst.atj_bst_utils import feed_parameters, dehyd_data, olig_data, prod_selectivity, hydgn_data, price_data
from atj_saf.atj_bst.atj_bst_units import AdiabaticReactor, IsothermalReactor, EthanolStorageTank, HydrocarbonProductTank, HydrogenStorageTank
from atj_saf.atj_bst.atj_bst_tea_saf import ConventionalEthanolTEA

In [3]:
atj_chems = create_chemicals()
bst.settings.set_thermo(atj_chems) # Setting thermodynamic property pacakge for the chemicals


In [4]:
bst.settings.CEPCI = 800.8 # For the year 2023 from https://personalpages.manchester.ac.uk/staff/tom.rodgers/Interactive_graphs/CEPCI.html?reactors/CEPCI/index.html

In [5]:
etoh_flow = 7485.07 # [kg/hr] Just adding this here, This will change once model is integrated with Kays 

In [6]:
# Bioethanol feed
etoh_in = bst.Stream(
    'etoh_in',
    Ethanol = etoh_flow,
    Water =  etoh_flow*((1-feed_parameters['purity'])/(feed_parameters['purity'])),
    units = 'kg/hr',
    T = feed_parameters['temperature'],
    P = feed_parameters['pressure'],
    phase = feed_parameters['phase'])




In [7]:
 # Reactions

#1) Gas phase dehydration of ethanol to ethylene 
dehydration_rxn = bst.Reaction('Ethanol,g -> Water,g + Ethylene,g', reactant = 'Ethanol', 
                            X = dehyd_data['conv'], phases = 'lg',  basis = 'mol')


#2) Ethylene oligomerization to olefins in gas and liquid phase
oligomerization_rxn = bst.ParallelReaction([
# Reaction definition                                     # Reactant                    # Conversion
bst.Reaction('2Ethylene,g -> Butene,g',            reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C4H8'],    basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('1.5Ethylene,g -> Hex-1-ene,g',       reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C6H12'],   basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('5Ethylene,g -> Dec-1-ene,l',         reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C10H20'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('9Ethylene,g -> Octadec-1-ene,l',     reactant = 'Ethylene',     X = olig_data['conv']*prod_selectivity['C18H36'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True)])


hydrogenation_rxn = bst.ParallelReaction([
# Reaction definition                                           # Reactant                    # Conversion
bst.Reaction('Butene,g + Hydrogen,g -> Butane,g',               reactant = 'Butene',          X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Butene,l + Hydrogen,g -> Butane,l',               reactant = 'Butene',          X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Hex-1-ene,g + Hydrogen,g -> Hexane,g',            reactant = 'Hex-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Hex-1-ene,l + Hydrogen,g -> Hexane,l',            reactant = 'Hex-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Dec-1-ene,l + Hydrogen,g -> Decane,l',            reactant = 'Dec-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Dec-1-ene,g + Hydrogen,g -> Decane,g',            reactant = 'Dec-1-ene',       X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Octadec-1-ene,l + Hydrogen,g -> Octadecane,l',    reactant = 'Octadec-1-ene',   X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Octadec-1-ene,g + Hydrogen,g -> Octadecane,g',    reactant = 'Octadec-1-ene',   X = hydgn_data['conv'],  basis = 'mol',  phases = 'lg',  correct_atomic_balance = True)])


In [8]:
# Recycle streams
dehyd_recycle = bst.MultiStream('dehyd_recycle', phases = ('g','l'))         # Unreacted ethanol
ethylene_recycle = bst.MultiStream('ethylene_recycle', phases = ('g','l'))   # Unreacted ethylene   
h2_recycle= bst.Stream(ID = 'h2_recycle', P = 3e6, phase = 'g')            # Excess hydrogen


In [9]:
etoh_storage = EthanolStorageTank(ins = etoh_in)
etoh_storage.simulate()


pump_1 = bst.units.Pump('PUMP1', ins = etoh_storage.outs[0], P = 1373000)    
pump_1.simulate()

furnace_1 = bst.units.HXutility('FURNACE_1', ins = pump_1.outs[0], T = 500, rigorous = True)
furnace_1.simulate()

mixer_1 = bst.units.Mixer('MIXER_1', ins = (furnace_1.outs[0], dehyd_recycle), rigorous = True)
mixer_1.simulate()

furnace_2 =  bst.units.HXutility('FURNACE_2', ins = mixer_1.outs[0], T = 481 + 273.15, rigorous = True)
furnace_2.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Total Cost', has no defined bare-module factor in the 'EthanolStorageTank.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "


In [10]:
dehyd_1 = AdiabaticReactor('DEHYD_1', ins = furnace_2.outs[0],
                        conversion = dehyd_data['conv'],
                        temperature = dehyd_data['temp'],
                        pressure = dehyd_data['pressure'],
                        WHSV = dehyd_data['whsv'],
                        catalyst_price=price_data['dehydration_catalyst'],
                        catalyst_lifetime = dehyd_data['catalyst_lifetime'],
                        reaction = dehydration_rxn)
dehyd_1.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'AdiabaticReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Platform and ladders', has no defined bare-module factor in the 'AdiabaticReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Catalyst loading cost', has no defined bare-module factor in the 'AdiabaticReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "


In [11]:
dehyd_1.results()

Adiabatic reactor                              Units   DEHYD_1
Design              Vessel type                       Vertical
                    Length                        ft      25.3
                    Diameter                      ft      8.44
                    Weight                            2.76e+04
                    Wall thickness                in     0.803
                    Catalyst Weight               kg  2.51e+04
                    Volume                         L  4.01e+04
                    Duty                       kJ/hr         0
Purchase cost       Vertical pressure vessel     USD  2.67e+05
                    Platform and ladders         USD  2.75e+04
                    Catalyst loading cost        USD  9.23e+05
Total purchase cost                              USD  1.22e+06
Utility cost                                  USD/hr         0

In [12]:
# Dehydration catalyst replacement stream
syndol_wt = dehyd_1.get_design_result('Catalyst Weight', 'kg')
syndol_req = syndol_wt/dehyd_data['catalyst_lifetime']
syndol_replacement = bst.Stream(Syndol = syndol_req, phase = 's', units = 'kg/yr' )

In [13]:
splitter_1 = bst.units.Splitter(ins = dehyd_1.outs[0], outs = ('flash_in', dehyd_recycle), split = 0.3)
splitter_1.simulate()

flash_1 = bst.units.Flash('FLASH_1', ins = splitter_1.outs[0], outs = ('ETHYLENE_WATER', 'WW_1'), T= 420,  P = 1.063e6)
flash_1.simulate()


comp_1 = bst.units.IsentropicCompressor('COMP_1', ins = flash_1.outs[0], P = 2e6, vle = True, eta = 0.72, driver_efficiency = 1)
comp_1.simulate()   

distillation_1 = bst.units.BinaryDistillation('DISTILLATION_1', ins = comp_1.outs[0], 
                                            outs = ('ethylene_water', 'WW'),
                                LHK = ('Ethylene', 'Water'), 
                                P = 2e+06,
                                y_top = 0.999, x_bot = 0.001, k = 2,
                                is_divided = True)
# distillation_1.check_LHK = False   # Does not check for volatile components that might show up in lights
distillation_1.simulate()

comp_2 = bst.units.IsentropicCompressor('COMP_2', ins = distillation_1.outs[0], P = olig_data['pressure'], vle = True, eta = 0.72, driver_efficiency = 1)
comp_2.simulate()

distillation_2 = bst.units.BinaryDistillation('DISTILLATION_2', ins = comp_2.outs[0],
                                LHK = ('Ethylene', 'Ethanol'),
                                P = 3.5e+06,
                                y_top = 0.9999, x_bot = 0.0001, k = 2,
                                is_divided = True)
distillation_2.simulate()

cooler_1 = bst.units.HXutility('COOLER_1', ins = distillation_2.outs[1], outs = 'WW_2', T = 300, rigorous = True)
cooler_1.simulate()

splitter_2 = bst.units.Splitter('SPLIT2', ins = distillation_1.outs[1], split = 0.6)
splitter_2.simulate()



cooler_2 = bst.units.HXutility('COOLER_2', ins = splitter_2.outs[0], outs = 'WW_3', T = 300, rigorous = True)
cooler_2.simulate()

cooler_3 = bst.units.HXutility('COOLER_3', ins = distillation_2.outs[0], T = 393.15, rigorous = True)
cooler_3.simulate()

mixer_2 = bst.units.Mixer(ID = 'MIXER_3', ins = (cooler_3.outs[0],ethylene_recycle), rigorous = True)
mixer_2.simulate()


c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Ethylene has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel weight (314.6 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel length (5.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:1241: CostWarning: <IsentropicCompressor: COMP_2> power (37.22 hp) is out of bounds (200 to 3e+04 hp) for cost correlation
  self._cost(**cost_kwargs) i

In [14]:
atj_chems

CompiledChemicals([Water, Ethanol, AceticAcid, Furfural, Glycerol, H2SO4, NH3, LacticAcid, SuccinicAcid, P4O10, Lime, HNO3, NH4OH, Denaturant, DAP, AmmoniumAcetate, AmmoniumSulfate, NaNO3, Oil, HMF, N2, O2, CH4, H2S, SO2, CO2, NO2, NO, CO, Glucose, Xylose, Sucrose, CaSO4, Mannose, Galactose, Arabinose, CellulaseNutrients, Extract, Acetate, Tar, Ash, NaOH, Lignin, SolubleLignin, GlucoseOligomer, GalactoseOligomer, MannoseOligomer, XyloseOligomer, ArabinoseOligomer, Z_mobilis, T_reesei, Biomass, Cellulose, Protein, Enzyme, Glucan, Xylan, Xylitol, Cellobiose, CSL, DenaturedEnzyme, Arabinan, Mannan, Galactan, WWTsludge, Cellulase, Ethylene, Butene, Hex-1-ene, Dec-1-ene, Octadec-1-ene, Butane, Hexane, Octane, Decane, Octadecane, Hydrogen, Syndol, Nickel_SiAl, CobaltMolybdenum])


In [15]:
olig_1 = IsothermalReactor('OLIG_1', ins = mixer_2.outs[0],
                            conversion = olig_data['conv'],
                            temperature = olig_data['temp'],
                            pressure = olig_data['pressure'],
                            WHSV = olig_data['whsv'],
                            catalyst_price = price_data['oligomerization_catalyst'],
                        reaction = oligomerization_rxn)
olig_1.simulate()
# Oligomerization catalyst replacement stream


c:\users\hwadg\onedrive - the pennsylvania state university\shi_wadgama_shared\models\atjspk\atj_saf\atj_bst\atj_bst_units.py:224: CostWarning: <IsothermalReactor: OLIG_1> Vertical vessel weight (3213 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
c:\users\hwadg\onedrive - the pennsylvania state university\shi_wadgama_shared\models\atjspk\atj_saf\atj_bst\atj_bst_units.py:224: CostWarning: <IsothermalReactor: OLIG_1> Vertical vessel length (8.367 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'IsothermalReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: t

In [16]:
# Oligomerization catalyst replacement
ni_si_al_wt = olig_1.get_design_result('Catalyst Weight', 'kg')
ni_si_al_req = ni_si_al_wt/olig_data['catalyst_lifetime']
ni_si_al_replacement = bst.Stream(Nickel_SiAl = ni_si_al_req, phase = 's', units = 'kg/yr' )

In [17]:
splitter_3 = bst.units.Splitter('SPLIITER_3', ins = olig_1.outs[0], outs = (ethylene_recycle,'oligs'),  split = {'Ethylene':1.0})
splitter_3.simulate()

h2_in = bst.Stream(ID = 'h2_in',  P = 3e6, phase= 'g')
mixer_3 = bst.units.Mixer('mix_try', ins = (h2_in, h2_recycle), rigorous = True)
@mixer_3.add_specification(run = True)
def h2_flow():
    h2_flow = 3*((olig_1.outs[0].imol['Butene'] + olig_1.outs[0].imol['Hex-1-ene']
                    + olig_1.outs[0].imol['Dec-1-ene'] + olig_1.outs[0].imol['Octadec-1-ene']))
    
    h2_in.imol['Hydrogen'] = h2_flow - h2_recycle.imol['Hydrogen']
mixer_3.simulate()

h2_storage = HydrogenStorageTank('H2_STORAGE',ins = mixer_3.outs[0])
h2_storage.simulate()


mixer_4 = bst.units.Mixer(ins = (h2_storage.outs[0], splitter_3.outs[1]), rigorous = True)
mixer_4.simulate()


cooler_4 = bst.units.HXutility('COOLER_4', ins = splitter_2.outs[1], outs = 'WW_4', T = 300, rigorous = True)
cooler_4.simulate()

furnace_3 = bst.units.HXutility('FURNACE_3', mixer_4.outs[0], T = 350 +273.15, rigorous = True)
furnace_3.simulate()



c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Total Cost', has no defined bare-module factor in the 'HydrogenStorageTank.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\bubble_point.py:128: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)


In [18]:
hydgn_1 = AdiabaticReactor('hydgn', ins = furnace_3.outs[0],
                        conversion = hydgn_data['conv'],
                        temperature = hydgn_data['temp'],
                        pressure = hydgn_data['pressure'],
                        WHSV = hydgn_data['whsv'],
                        catalyst_price = price_data['hydrogenation_catalyst'],
                        reaction = hydrogenation_rxn)
hydgn_1.simulate()

c:\users\hwadg\onedrive - the pennsylvania state university\shi_wadgama_shared\models\atjspk\atj_saf\atj_bst\atj_bst_units.py:92: CostWarning: <AdiabaticReactor: hydgn> Vertical vessel weight (1749 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
c:\users\hwadg\onedrive - the pennsylvania state university\shi_wadgama_shared\models\atjspk\atj_saf\atj_bst\atj_bst_units.py:92: CostWarning: <AdiabaticReactor: hydgn> Vertical vessel length (6.755 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'AdiabaticReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purc

In [19]:
# Hydrogenation catalyst replacement
co_mo_wt = hydgn_1.get_design_result('Catalyst Weight', 'kg')
co_mo_req = co_mo_wt/hydgn_data['catalyst_lifetime']
co_mo_replacement = bst.Stream(CobaltMolybdenum = co_mo_req, phase = 's', units = 'kg/yr' )

In [20]:
cooler_5 = bst.units.HXutility('COOLER_5', ins = hydgn_1.outs[0], T = 250, rigorous = True)
cooler_5.simulate()
flash_2 = bst.units.Flash(ins = cooler_5-0, T = 250, P = 5e5, outs = (h2_recycle, 'fuel'))
flash_2.simulate()

distillation_3 = bst.units.BinaryDistillation('DISTILLATION_3', ins = flash_2.outs[1],
                                outs = ('distillate', 'bottoms'),
                                LHK = ('Hexane', 'Decane'),
                                y_top = 0.99, x_bot = 0.01, k = 2,
                                is_divided = True)
#distillation_3.check_LHK = False
distillation_3.simulate()

distillation_4 = bst.units.BinaryDistillation('DISTILLATION_4', ins = distillation_3.outs[1],
                                outs = ('distillate_1', 'bottoms_1'),
                                LHK = ('Decane', 'Octadecane'),
                                y_top = 0.99, x_bot = 0.01, k = 2,
                                is_divided = True)
distillation_4.simulate()

cooler_6 = bst.units.HXutility('COOLER_6', ins = distillation_3.outs[0]
                            ,V = 0, rigorous = True)
cooler_6.simulate()


cooler_7 = bst.units.HXutility('COOLER_7', ins = distillation_4.outs[0],T = 15+273.15, rigorous = True)
cooler_7.simulate()
cooler_7.outs[0].phase = 'l'   # Just setting it as liquid, because rigorous = True gives both l and g phases. 
                               # Actual phase is liquid only so asserting phase like this is fine

cooler_8 = bst.units.HXutility('COOLER_8', ins = distillation_4.outs[1],T = 15+273.15, rigorous = True)
cooler_8.simulate()
cooler_8.outs[0].phase = 'l'


rn_storage = HydrocarbonProductTank('RN_STORAGE', ins = cooler_6.outs[0], outs = 'RN')
rn_storage.simulate()

saf_storage = HydrocarbonProductTank('SAF_STORAGE', ins = cooler_7.outs[0], outs = 'SAF')
saf_storage.simulate()


rd_storage = HydrocarbonProductTank('RD_STORAGE', ins = cooler_8.outs[0], outs = 'RD')
rd_storage.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\equilibrium\dew_point.py:129: RuntimeWarning: Hydrogen has no defined Dortmund groups; functional group interactions are ignored
  self.gamma = thermo.Gamma(chemicals)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel weight (255.2 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel length (4.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Total Cost', has no defined bare-module factor in the 'HydrocarbonProductTank.F_BM' diction

In [21]:
atj_sys = bst.System('atj_sys', path = (etoh_storage, pump_1, furnace_1, mixer_1, furnace_2, dehyd_1, splitter_1, flash_1, comp_1, 
                                        distillation_1, comp_2, distillation_2, cooler_1, splitter_2, cooler_2, cooler_3, mixer_2,
                                        olig_1, splitter_3, mixer_3, h2_storage, mixer_4, cooler_4, furnace_3, hydgn_1, cooler_5, 
                                        flash_2, distillation_3, distillation_4, cooler_6, cooler_7, cooler_8,
                                        rn_storage, saf_storage, rd_storage), 
                                        recycle = (dehyd_recycle, ethylene_recycle, h2_recycle))
atj_sys.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel weight (860.9 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <Flash: FLASH_1> Vertical vessel length (6.5 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:1241: CostWarning: <IsentropicCompressor: COMP_2> power (122.4 hp) is out of bounds (200 to 3e+04 hp) for cost correlation
  self._cost(**cost_kwargs) if cost_kwargs else self._cost()
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel weight (751.3 lb) is out of bounds (4200 to 

In [22]:
atj_sys.show()

System: atj_sys
Highest convergence error among components in recycle
streams {S1-1, SPLIITER_3-0, F1-0} after 4 loops:
- flow rate   4.99e-03 kmol/hr (0.62%)
- temperature 7.40e-04 K (0.00012%)
ins...
[0] etoh_in  
    phase: 'l', T: 293.15 K, P: 101325 Pa
    flow (kmol/hr): Water    2.09
                    Ethanol  162
[1] h2_in  
    phase: 'g', T: 298.15 K, P: 3e+06 Pa
    flow (kmol/hr): Hydrogen  44.9
outs...
[0] SAF  
    phases: ('g', 'l'), T: 288.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Ethanol     0.00811
                        Hexane      0.202
                        Decane      20
                        Octadecane  0.202
[1] RD  
    phases: ('g', 'l'), T: 288.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Decane      0.00341
                        Octadecane  0.338
[2] WW_1  
    phase: 'l', T: 420 K, P: 1.063e+06 Pa
    flow (kmol/hr): Water     51.5
                    Ethanol   0.00892
                    Ethylene  0.162
[3] RN  
    phase: 'l', T: 213.38 K, P: 1013

### Estimation of labor costs
In accordance with methodology in Sieder book

In [23]:

operators_per_section = 1  # operators per section from Seider recommendation
num_process_sections = 5  # number of proces sections from Seider recommendation [3 reactor, 2 separation]
num_operators_per_shift = operators_per_section * num_process_sections * 1  # multiplied by 2 for large continuous flow process (e.g., 1000 ton/day product). from Seider pg 505
num_shifts = 5  # number of shifts
pay_rate = 40  # $/hr
DWandB = num_operators_per_shift * num_shifts * 2080 * pay_rate  # direct wages and benefits. DWandB [$/year] = (operators/shift)*(5 shifts)*(40 hr/week)*(operating days/year-operator)*($/hr)
Dsalaries_benefits = 0.15 * DWandB  # direct salaries and benefits from Seider
O_supplies = 0.06 * DWandB  # Operating supplies and services from Seider
technical_assistance = 5 * 75000  # $/year. Technical assistance to manufacturing. assume 5 workers at $75000/year
control_lab = 5 * 80000  # $/year. Control laboratory. assume 5 workers at $80000/year
labor = DWandB + Dsalaries_benefits + O_supplies + technical_assistance + control_lab 

    

In [24]:
atj_tea = ConventionalEthanolTEA(system = atj_sys,
                                IRR = 0.10,
                                duration = (2023, 2053),
                                depreciation = 'MACRS7',
                                income_tax = 0.21,
                                operating_days = 330,
                                lang_factor = 5.04,
                                construction_schedule = (0.08, 0.60, 0.32),
                                WC_over_FCI = 0.05,
                                labor_cost = labor,
                                #fringe_benefits = 0,
                                property_tax=0.001, 
                                property_insurance=0.005, 
                                #supplies=0, 
                                maintenance=0.01, 
                                administration=0.005
                                )

### Prices data 
Will be updated to a yaml file in the future once model integration is finalized

In [25]:
atj_sys.feeds[0].price = 0.9    # Ethanol price, subject to change once integration with Kays model happens
atj_sys.feeds[1].price = price_data['hydrogen']
atj_sys.products[0].price = -price_data['wastewater_treatment']
atj_sys.products[1].price = -price_data['wastewater_treatment']
atj_sys.products[2].price = -price_data['wastewater_treatment']
atj_sys.products[3].price = -price_data['wastewater_treatment']
atj_sys.products[4].price = price_data['renewable_naphtha']
saf_stream = atj_sys.products[5] 
atj_sys.products[6].price = price_data['renewable_diesel']


In [26]:
print(f'The MSP is {round((atj_tea.solve_price(saf_stream)*saf_stream.rho)/264.172,2)} USD/gal')

The MSP is 8.72 USD/gal
